In [1]:
import os
import albumentations as A
from pathlib import Path
import cv2


In [3]:

img_folder_path = './dataset/images/'
annot_folder_path = './dataset/labels/'

# main function to augment images
def transformer(img_folder_path, annot_folder_path):
    
    img_paths = Path(img_folder_path).glob('*.jpg')   

    for img_path in img_paths:

        img_name = str(img_path).split('\\')[-1].split('.')[:-1]
        img_name = '.'.join(img_name)
        label_name = img_name + '.txt'
        
        label_path = annot_folder_path + label_name
                    
        result = horizontalFlip(str(img_path), label_path, img_name)      
           
    return 

   
def horizontalFlip(img_path, label_path, save_name):  
    
    # setup transformer
    transformHorizontalFlip = A.Compose([A.HorizontalFlip(always_apply=True)], 
                      bbox_params=A.BboxParams(format='yolo'))  
    
    # annotation file to list
    with open(label_path) as f:
        lines = f.readlines()

    bbox = []
    for line in lines:
        l = line.split(' ')
        bbox.append([float(l[1]), float(l[2]), float(l[3]), float(l[4]), str(l[0])])
    
    img = cv2.imread(img_path) 
    
    try:
        trns = transformHorizontalFlip(image = img, bboxes=bbox)
    
        # transformation results
        img_transformed = trns['image']   
        boxes = trns['bboxes']
        
        # save image and annotation file in YOLO format 
        cv2.imwrite('./dataset/images/'+ save_name + '_hf.jpg', img_transformed)    
        
        with open(r'dataset/labels/'+ save_name + '_hf.txt' , 'w') as f:     
            for box in boxes:
                f.write(str(box[-1]))
                f.write(' '+str(box[0]))
                f.write(' '+str(box[1]))
                f.write(' '+str(box[2]))
                f.write(' '+str(box[3]))
                f.write('\n')
            
        return 
    
    except:
        print('Failed:', save_name)
        return 


transformer(img_folder_path, annot_folder_path)

In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Basispaden definiëren
base_dir = './dataset'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# Bestemmingsmappen aanmaken
train_images_dir = os.path.join(base_dir, 'train/images')
train_labels_dir = os.path.join(base_dir, 'train/labels')
val_images_dir = os.path.join(base_dir, 'val/images')
val_labels_dir = os.path.join(base_dir, 'val/labels')
test_images_dir = os.path.join(base_dir, 'test/images')
test_labels_dir = os.path.join(base_dir, 'test/labels')

# Mappenstructuur aanmaken
for path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, test_images_dir, test_labels_dir]:
    os.makedirs(path, exist_ok=True)

# Lijsten van afbeeldingen en labels verzamelen
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

# Zorg ervoor dat de bestanden overeenkomen en in dezelfde volgorde staan
image_files.sort()
label_files.sort()

# Dataset splitsen
# Eerste splitsing: 80% train, 20% temp (voor val en test)
images_train, images_temp, labels_train, labels_temp = train_test_split(image_files, label_files, test_size=0.2, random_state=42)

# Tweede splitsing: temp in 50% val en 50% test
images_val, images_test, labels_val, labels_test = train_test_split(images_temp, labels_temp, test_size=0.5, random_state=42)

# Kopieerfunctie om bestanden te verplaatsen
def copy_files(files, source_dir, destination_dir):
    for file in files:
        shutil.move(os.path.join(source_dir, file), os.path.join(destination_dir, file))

# Bestanden naar de juiste mappen verplaatsen
copy_files(images_train, images_dir, train_images_dir)
copy_files(labels_train, labels_dir, train_labels_dir)
copy_files(images_val, images_dir, val_images_dir)
copy_files(labels_val, labels_dir, val_labels_dir)
copy_files(images_test, images_dir, test_images_dir)
copy_files(labels_test, labels_dir, test_labels_dir)


In [5]:
import torch
from ultralytics import YOLO

model = YOLO("yolov8x.pt") # load a pretrained model 

# Train the model
results = model.train(data="../path-of-yaml/licence_plate.yaml", 
      name = 'licence_plate', epochs=100, patience=20, batch = 16, 
      cache = True, imgsz=640, iou = 0.5,
      augment=True, degrees=25.0, fliplr=0.0, lr0=0.0001, optimizer='Adam')